### Startup

This code is meant to be executed on Google Colab.
To use it locally change *COLAB_MODE* to False.

**Note**: remember to change *workdir* accordingly, the notebook must be runned inside the root project folder

In [2]:
workdir = "./src"
%cd $workdir

/home/fra/AIGCDetection-CI-CD/src


/home/fra/miniconda3/envs/AIGCdetection/lib/python3.12/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [5]:
!python train.py -c ../model_config.conf \
    --input_model ../checkpoints/model.pth \
    --output_filepath  ../checkpoints/test/best_acc.pth \
    --target_dir ../datasets/biggan \
    --source_datasets ../datasets/cyclegan

Namespace(config_file='../model_config.conf', input_model='../checkpoints/model_best_accuracy.pth', output_filepath='../checkpoints/test/best_acc.pth', target_dir='../datasets/biggan', source_datasets='../datasets/cyclegan', num_gpu='0', network='ResNet', epochs='1', early_stop='25', batch_size='64', resolution='128', lr_schedule='cosine', lr='0.005', kd_alpha='0.5', decay_factor='0.9', flip='False')



------ Creating Loaders ------
GPU num is 0

===> Making Loader for Continual Learning..
===> Making Loader : ../datasets/cyclegan
DATASET PATHS
val_source_dir  ../datasets/cyclegan
val_target_dir  ../datasets/biggan/val
train_dir  ../datasets/biggan/train
Dataset available in train_loaders:  train / val
Dataset available in val_loaders:  ../datasets/cyclegan



 ------ Loading models ------
Loading ResNet from ../checkpoints/model_best_accuracy.pth
Loaded
Apply Cosine learning rate schedule
Adjusting learning rate of group 0 to 5.0000e-03.
Start training in 1 epochs
/home/fra/miniconda

## Evaluation

In [4]:
import json
import base64

!python tests/test_client.py --input_model ../checkpoints


{"image": "...\nImRu/w8wseTK1gRkdAAAAABJRU5ErkJggg==\n"}
Namespace(input_model='../checkpoints', raw_data=None)
{"image": "...\n2ajXE/8PIRhl4tZq1aYAAAAASUVORK5CYII=\n"}
tensor([[     0.0000, -34982.8359]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -25096.5508]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -43666.8125]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -29370.9258]]) tensor([0])
{"image": "...\nSeV0+f8AZmF8G7kiaGMAAAAASUVORK5CYII=\n"}
tensor([[     0.0000, -19328.7656]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -36587.0859]]) tensor([0])
{"image": "...\nRK5CYII=\n"}
tensor([[     0.0000, -30430.0234]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -30722.3125]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -32362.0625]]) tensor([0])
{"image": "...\nsNT/P8Y4ARD9GZAxAAAAAElFTkSuQmCC\n"}
tensor([[     0.0000, -34646.0234]]) tensor([0])
{"image": "...\n"}
tensor([[     0.0000, -35033.9531]]) tensor([0])
{"image": "...\n"}
t